In [1]:
module Compositions

compositions = (;
    co2impure="REFPROP::CarbonDioxide[0.969696970]&Hydrogen[0.010101010]&Nitrogen[0.010101010]&Argon[0.009090909]&CarbonMonoxide[0.001010101]",
    co2pure="CarbonDioxide[1.0]"
)

export compositions

end

Main.Compositions

In [2]:
using .Compositions, CoolProp, DifferentialEquations, ModelingToolkit, Unitful

@variables t

1-element Vector{Num}:
 t

# next section

In [3]:

@register_symbolic CoolProp.:PropsSI(out::AbstractString, name1::AbstractString, value1::Real, name2::AbstractString, value2::Real, fluid::AbstractString)


const gn = Unitful.gn.val

function _NodeEnergy(node, D, ρ)
    return node.p / (ρ * gn) + node.z + 8 * abs2(node.q) / (π^2 * D^4 * gn)
end

"""
To get the loss of resistance along the pipe(between two ports).

In volume flow form:

```math
h_f = f\\frac{L}{D} \\frac{8q^{2}}{\\pi^2D^4g}
```

"""
function _h_f(node, f, L, D)
    return f * L * 8 / (D^5 * π^2 * gn) * node.q * abs(node.q)
end


"""
To get the local resistance loss the components.

In volume flow form:

```math
h_m = K \\frac{8q^{2}}{\\pi^2D^4g}
```

"""
function _h_m(node, K, D)
    return K * 8 / (D^4 * π^2 * gn) * node.q * abs(node.q)
end

function Sink_P(; name, p=101325)
    @named port = PipeNode2(z=0.0)
    eqs = [
        port.p ~ p
    ]
    compose(ODESystem(eqs, t, [], [], name=name), port)
end

@connector function PipeNode2(; name, z=0)
    sts = @variables begin
        p(t) = 1.0
        (q(t)=1.0, [connect = Flow])
    end
    ps = @parameters z = z
    ODESystem(Equation[], t, sts, ps; name=name)
end

function Pipe(; name, P=1.013e5, Tm=300, L=10.0, D=1.0, f=0.01, K_inside=0.0)
    @named in = PipeNode2()
    @named out = PipeNode2()
    ps = @parameters L=L D=D f=f K_inside=K_inside u=20
    # @variables t x(t) y(t) z(t)
    @variables begin
        (qm=0, [connect = Flow])
        # (qv=0, [connect = Stream])
        Aₛ
        p = P
        tm(t) = Tm
        ρ
        μ
    end
    eqs = [
        Aₛ ~ π * D * L,
        ρ ~ PropsSI("D", "P", p, "T", Tm, compositions.co2pure),
        p ~ P,
        μ ~ PropsSI("V", "P", p, "T", Tm, compositions.co2pure),
        # qv ~ qm / ρ,
        _NodeEnergy(in, D, ρ) ~ _NodeEnergy(out, D, ρ) + _h_f(in, f, L, D) + _h_m(in, K_inside, D) + u * Aₛ,
        0 ~ in.q + out.q
        ]
    
    ODESystem(eqs, t, [], ps; name=name)
    compose(ODESystem(eqs, t, [], ps, name=name), in, out)
end

Pipe (generic function with 1 method)

In [4]:
@named test = Pipe()

Model test with 6 (8) equations
States (4):
  in₊p(t) [defaults to 1.0]
  in₊q(t) [defaults to 1.0]
  out₊p(t) [defaults to 1.0]
  out₊q(t) [defaults to 1.0]
Parameters (7):
  L [defaults to 10.0]
  D [defaults to 1.0]
  f [defaults to 0.01]
  K_inside [defaults to 0.0]
⋮

In [5]:
@named low = Sink_P(p=100000)

Model low with 1 (2) equations
States (2):
  port₊p(t) [defaults to 1.0]
  port₊q(t) [defaults to 1.0]
Parameters (1):
  port₊z [defaults to 0.0]

In [6]:
eqs = [connect(test.out, low.port)]

1-element Vector{Equation}:
 connect(test.out, low.port)

In [7]:
@named model = compose(ODESystem(eqs, t, name=:funs), [test, low])


Model model with 7 (10) equations
States (6):
  test₊in₊p(t) [defaults to 1.0]
  test₊in₊q(t) [defaults to 1.0]
  test₊out₊p(t) [defaults to 1.0]
  test₊out₊q(t) [defaults to 1.0]
⋮
Parameters (8):
  test₊L [defaults to 10.0]
  test₊D [defaults to 1.0]
  test₊f [defaults to 0.01]
  test₊K_inside [defaults to 0.0]
⋮

In [8]:
sys = structural_simplify(model)
prob = ODEProblem(sys, [], (0.0, 0.0))
sol = ModelingToolkit.:solve(prob)

retcode: Success
Interpolation: specialized 4rd order "free" stiffness-aware interpolation
t: 1-element Vector{Float64}:
 0.0
u: 1-element Vector{Vector{Float64}}:
 [111067.5874933406]

In [13]:
sol[test.in.p]

1-element Vector{Float64}:
 111067.5874933406

In [21]:
Dict("retcode" => sol.retcode, "t" => sol.t)

Dict{String, Any} with 2 entries:
  "t"       => [0.0]
  "retcode" => Success

In [22]:
String(Dict("retcode" => sol.retcode, "t" => sol.t))

MethodError: MethodError: no method matching String(::Dict{String, Any})

Closest candidates are:
  String(!Matched::CommonMark.AbstractParser)
   @ CommonMark ~/.julia/packages/CommonMark/eJY7b/src/parsers.jl:6
  String(!Matched::String)
   @ Core boot.jl:360
  String(!Matched::Core.Compiler.LazyString)
   @ Core strings/lazy.jl:80
  ...


In [23]:
using JSON3

In [26]:
JSON3.write(Dict("retcode" => sol.retcode, "t" => sol.t, "u" => sol.u))

"{\"t\":[0.0],\"u\":[[111067.5874933406]],\"retcode\":\"Success\"}"